In [1]:
import pandas as pd
import numpy as np
import math
from bridge_specs import box
from Cross import calc_centroid as cg, calc_moi as moi
from irc6_2007 import *

In [2]:
#inputs

df = pd.read_excel('outputs/loads.xlsx', index_col=[0, 1])
display(df)
ll = df.loc[[('MaxSF','Class70RW')]].max().max()

0.00      6.25      12.50      18.75      25.00  \
MaxBM  ClassA       0.000  2361.037  4068.075   5120.737   5526.150   
       Class70RW    0.000  4826.025  8200.950  10182.475  10877.400   
       Class70RT    0.000  3651.438  6259.675   7823.212   8342.050   
MaxSF+ ClassA     445.318   376.622   307.926    238.122    169.426   
       Class70RW  895.524   771.524   647.524    521.524    397.524   
       Class70RT  667.590   579.390   492.590    404.390    317.590   
MaxSF- ClassA       0.000   -12.030   -51.234   -106.602   -175.162   
       Class70RW    0.000   -17.024   -87.536   -208.476   -332.476   
       Class70RT    0.000   -54.810  -143.010   -229.810   -316.610   
MaxSF  ClassA     445.318   376.622   307.926    238.122    175.162   
       Class70RW  895.524   771.524   647.524    521.524    397.524   
       Class70RT  667.590   579.390   492.590    404.390    317.590   

                      31.25     37.50     43.75    50.00  
MaxBM  ClassA      5340.038  4327.675  2494.638    0.000  
       Class70RW  10163.225  8053.750  4611.975    0.000  
       Class70RT   7823.588  6260.425  3652.563    0.000  
MaxSF+ ClassA        99.676    43.520    13.056    0.000  
       Class70RW    271.524   148.964    44.948    0.000  
       Class70RT    229.390   142.590    54.390    0.000  
MaxSF- ClassA      -244.966  -313.662  -383.466 -452.162  
       Class70RW   -458.476  -582.476  -708.476 -832.476  
       Class70RT   -404.810  -491.610  -579.810 -666.610  
MaxSF  ClassA       244.966   313.662   383.466  452.162  
       Class70RW    458.476   582.476   708.476  832.476  
       Class70RT    404.810   491.610   579.810  666.610

In [3]:
def loader(unit_wt, length, width, height, type):
    if type == 'r':
        load = unit_wt*length*width*height
    elif type == 't':
        load = unit_wt*length*width*height*0.5
    
    return load
def z(base, clr, dim, x, type):
    """base = width of base along with taper
       clr = overhang part from where centroid is calculated
        dim = width of object whose lever arm is required
        type = shape of object
    """
    y = []
    if type == 'r':
        y.append(cg('rectangle',[dim,0],[x,0]))
        return (-(clr+base/2)+y[0][0])
    y = []
    if type == 't1':
        y.append(cg('triangle_2',[dim,0],[x,0]))
        return (-(clr+base/2)+y[0][0])
    y = []
    if type == 't2':
        y.append(cg('triangle_1',[dim,0],[x,0]))
        return (-(clr+base/2)+y[0][0])

def na(x,s,cf,dm): 
    mor = (s*cf)/1000
    while dm>=mor:
        x = int(input("Enter a new trial depth:"))
        cs = d-0.416*xu
        cf = (0.446*fck*beta1*(1000*abut['width_of_abut'])*xu)/1000 
        mor = (s*cf)/1000
    return x

Abutment details

In [66]:
#inputs
df=pd.read_excel('data/abutment.xlsx',index_col=None,header=None)
df2=pd.read_excel('data/abutment2.xlsx',index_col=None,header=None)
display(df2)
bearing_ht = 0.3 #assumed including pedestal
abut = {
    'total_ht' :df.iloc[1,6]+df.iloc[0,2]+df.iloc[1,8],
    'wc' : 0.075,
    'backw_ht' :df.iloc[0,2], #including wearing cource
    'cap':df.iloc[1,4],
    'stem_ht': df.iloc[1,6],
    'bw_width':df.iloc[1,2],
    'edge_to_bearing_c':df.iloc[0,9],
    'bw_to_bearing_c': df.iloc[1,9],
    'cap_width':df.iloc[0,4],
    'stem_width':df.iloc[0,6],
    'be_taperwidth':df.iloc[0,7],
    'width_of_abut':df2.iloc[1,0],
    'ttl_stembase_width':df2.iloc[0,0],
    'approach_slab_dep':df.iloc[1,8],
    'approach_slab_len':df.iloc[0,8],
    'haunch':{'stem_haunch':[df.iloc[0,5],df.iloc[1,5]],'approach_slab_haunch_rec':[df.iloc[0,0],df.iloc[1,0]], 'approach_slab_haunch_tri':[df.iloc[0,1],df.iloc[1,1]]} #list of width and height.
}
fdn = {
    'heel' : df2.iloc[3,0],
    'toe' : df2.iloc[2,0],
    'th' : df2.iloc[4,0],
    'wid' : df2.iloc[5,0],
    'len' : df2.iloc[2,0]+df2.iloc[3,0]+df2.iloc[0,0]
}
# print(abut['stem_ht'])

,0
0,0.00
1,1.35
2,8.40
3,2.50
4,4.00
5,1.50
6,9.00
7,1.35
8,8.40
9,2.50


Inputs: Live Loads, Dead loads and Pedestrian loads

In [5]:
#dummy loads
df1=pd.read_csv('outputs/DL_for_Seismic.csv',index_col=None,header=None)
print(df1)
dl = float(df1.iloc[1,2]) + float(df1.iloc[2,2]) #dead loads
# ll = 1000 #live loads including impact
pdl = float(df1.iloc[4,2]) #pedestrian loads
sdl = float(df1.iloc[3,2]) #Surface Loads
print(type(pdl))

                 0                  1                  2                  3
0              NaN                 RL                 RR                Sum
1        Dead Load  4895.625000000001  4895.625000000001  9791.250000000002
2       Other Load              325.0              325.0              650.0
3     Surface Load              214.5              214.5              429.0
4  Pedestrian load              150.0              150.0              300.0
<class 'float'>


Input: Material properties

In [6]:
#Inputs


df1=pd.read_excel('data/material.xlsx',index_col=None,header=None)
# print(df1)
E = 200000 #Elasticity of steel
fck = df1.iloc[1,0] #grade of concrete = M30
fy = df1.iloc[2,0]#grade of steel = Fe500
unitwt_conc = 25 #unit weight of concrete = 25KN/m^3
unitwt_bfill = 20 #unit weight of backfill = 20KN/m^3

Self weight loads of abutment

In [7]:
# comp = ['stem_rect','stem_tri','cap','backwall','stem_haunch','approach_slab_haunch']
loads = [loader(unitwt_conc,abut['stem_ht'],abut['width_of_abut'],abut['stem_width'],'r'),
         loader(unitwt_conc,abut['stem_ht'],abut['width_of_abut'],abut['be_taperwidth'],'t'),
         loader(unitwt_conc,abut['cap'],abut['width_of_abut'],abut['cap_width'],'r'),
         loader(unitwt_conc,abut['backw_ht'],abut['width_of_abut'],abut['bw_width'],'r'),
         loader(unitwt_conc,abut['haunch']['stem_haunch'][0],abut['width_of_abut'],abut['haunch']['stem_haunch'][1],'t'),
         loader(unitwt_conc,abut['haunch']['approach_slab_haunch_rec'][0],abut['width_of_abut'],abut['haunch']['approach_slab_haunch_rec'][1],'r'),
          loader(unitwt_conc,abut['haunch']['approach_slab_haunch_tri'][0],abut['width_of_abut'],abut['haunch']['approach_slab_haunch_tri'][1],'t')]
lever_arm = [round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['stem_width'],abut['haunch']['stem_haunch'][0],'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['be_taperwidth'],abut['haunch']['stem_haunch'][0]+abut['stem_width'],'t1'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['cap_width'],0,'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['bw_width'],0,'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['haunch']['stem_haunch'][0],0,'t2'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],-abut['haunch']['approach_slab_haunch_rec'][0],0,'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],-abut['haunch']['approach_slab_haunch_tri'][0],0,'t2'),5)]
moments = []
for index,i in enumerate(loads):
    moments.append(loads[index]*lever_arm[index])
A1 = pd.DataFrame({"Load":loads,"Lever arm":lever_arm,"Moment":moments},index = ['Stem(rectangle)','Stem(triangle)','Cap','Backwall','Stem haunch','Approach slab haunch(rec)','Approach slab haunch(tri)'])
A1.loc[len(A1.index)] = [A1['Load'].sum(),None,A1['Moment'].sum()]
display(A1.rename(index={7:'Total'}))
display(A1.loc[7,'Load'])

,Load,Lever arm,Moment
Stem(rectangle),12.571875,1.86250,23.415117
Stem(triangle),0.506250,3.82500,1.936406
Cap,1.687500,-3.60000,-6.075000
Backwall,28.096875,-3.57500,-100.446328
Stem haunch,78.574219,-1.24167,-97.563250
Approach slab haunch(rec),2.109375,-3.85000,-8.121094
Approach slab haunch(tri),1.054688,-3.89167,-4.104496
Total,124.600781,NaN,-190.958644


124.60078125000001

Loads from superstructure

In [8]:
load = [dl,ll,sdl]
print(load)
arm = [round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['edge_to_bearing_c']+abut['bw_to_bearing_c'],abut['bw_width'],'r'),5),
      round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['edge_to_bearing_c']+abut['bw_to_bearing_c'],abut['bw_width'],'r'),5),
      round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['edge_to_bearing_c']+abut['bw_to_bearing_c'],abut['bw_width'],'r'),5)]
moment = []
for index,i in enumerate(load):
    moment.append(load[index]*arm[index])
A2 = pd.DataFrame({'Load':load, 'Lever arm':arm,'Moment':moment}, index = ['Dead load','Live load','Surface load'])
display(A2)
# display(A2.iloc[1,0])

[5220.625000000001, 895.524, 214.5]


,Load,Lever arm,Moment
Dead load,5220.625,-3.175,-16575.484375
Live load,895.524,-3.175,-2843.288700
Surface load,214.500,-3.175,-681.037500


Earth Pressure as per clause 214 of IRC:6

In [9]:
#Inputs

phi = 33 #Angle of internal friction of soil
alpha = 0 #Angle which earth face of the wal makes with the vertical of the wall
beta = 0 #Slope of earth fill
delta = 2/3*phi if 2/3*phi<22.5 else 22.5 #Angle of friction between earth and earth fill
h = abut['total_ht']-abut['approach_slab_dep'] #Height of earthfill
print(delta)

22.0


In [10]:
ep = unitwt_bfill*h*ka(alpha, beta, delta, phi) #Earth pressure in KN/m^2.
print(ep)
force = 0.5*ep*abut['width_of_abut']*h #Force due to earth pressure in KN.
forc = [force*math.cos(math.radians(delta)),force*math.cos(math.radians(delta)),force*math.sin(math.radians(delta))] #Horizontal and vertical components.
larm = [0.42*h, 0.33*h, -(abut['stem_width']+abut['be_taperwidth'])/2 ] #0.42 times height of wall above base for dry soil and 0.33 for wet.
momen = []
for index,i in enumerate(forc):
    momen.append(forc[index]*larm[index])
A3 = pd.DataFrame({'Force(KN)':forc, 'Lever arm(m)':larm,'Moment(KNm)':momen}, index = ['Earth pressure Hz(dry)','Earth pressure Hz(wet)','Earth pressure Ver'])
display(A3)

15.206449999999998


,Force(KN),Lever arm(m),Moment(KNm)
Earth pressure Hz(dry),27.361211,1.20750,33.038663
Earth pressure Hz(wet),27.361211,0.94875,25.958949
Earth pressure Ver,11.054647,-2.01250,-22.247477


Braking Forces

In [11]:
#Inputs

wt_70r = 1000 #Weight of 70R vehicle in KN.
classa_wt = 554 #Weight of Class A in KN.
pt = 1.2 #Point of application of load above the deck in m.
factor = 0.2 #Lane fact as per clause 211.2 of IRC 6

In [12]:
F = [1*wt_70r*factor, 2*classa_wt*factor]
la = pt+abut['total_ht']
m = max(F)*la
A4 = pd.DataFrame({'Force(KN)':max(F), 'Lever arm(m)':la,'Moment(KNm)':m}, index = ['Braking load'])
display(A4)

,Force(KN),Lever arm(m),Moment(KNm)
Braking load,221.6,4.675,1035.98


Temperature Forces

In [13]:
temp_var = 30 #Temperature variation in degree celsius
alp = 0.000012 #Coefficient of thermal expansion in m/deg. c/m
l = box['span'] #span of bridge
shr_str = 0.0002 #Strain due to shrinkage.
th_el = temp_var*alp*l #Thermal elongation in m.
shr_el = l*shr_str #Shrinkage elongation in m.
tot = (th_el+shr_el)/2 #Total elongation in m.
shr_rt = 1000 #Shear rating of elastomer bearing in KN/m/m^2
arb = 0.235644 #Area of bearing in m^2.
htb = 0.0340 #Height of bearing in m.
no = 3 #Number of bearings.
f = 1.1*tot*(shr_rt*arb)/htb
# print(box['span'])

In [14]:
frc = 3*f
La = abut['stem_ht']+abut['cap']+bearing_ht
A5 = pd.DataFrame({'Force(KN)':frc, 'Lever arm(m)':La,'Moment(KNm)': (frc*La)}, index = ['Temperature force'])
display(A5)

,Force(KN),Lever arm(m),Moment(KNm)
Temperature force,320.198612,0.6,192.119167


Live Load Surcharge as per clause 214.1.1.3 of IRC 6

In [15]:
hl = 1.2 
Pr = ka(alpha, beta, delta, phi)*hl*unitwt_bfill #Horizontal pressure 
print(Pr)
A6 = pd.DataFrame({'Force(KN)':(Pr*abut['width_of_abut']*(abut['total_ht']-abut['approach_slab_dep'])), 'Lever arm(m)':abut['total_ht']/3,'Moment(KNm)':((Pr*abut['width_of_abut']*(abut['total_ht']-abut['approach_slab_dep']))*abut['total_ht']/3)}, index = ['Force due to LL surcharge'])
display(A6)


6.34704


,Force(KN),Lever arm(m),Moment(KNm)
Force due to LL surcharge,24.634449,1.158333,28.534903


Seismic loads as per clause 218 of IRC 6

In [16]:
"""Inputs"""
zf = zone_factor(5) 
d = 1000 #Seismic importance factor as per table 19 of IRC 6
i = 1.2
f = 250 # Average response acceleration coefficient (see clause 218.5.1)
rrf = 3.0 #Response reduction factor as per table 20 of IRC 6 
ahh = ah(zf,sag(vib_period(d,f),'I'),rrf,1.2)
av = ahh*2/3
lam = max(math.degrees(math.atan(ahh/(1+av))),math.degrees(math.atan(ahh/(1-av))))
print(lam)

11.560130794217766


Due to self weight and superstructure loads

In [17]:
H_force = []
for index, i in enumerate(loads):
    H_force.append(round(loads[index]*ahh,5))
ht = [cg('rectangle', [abut['stem_width'],abut['stem_ht']], [0,0])[1], 
      cg('triangle_2', [abut['be_taperwidth'],abut['stem_ht']], [0,0])[1],
      cg('rectangle', [abut['cap_width'],abut['cap']], [0,abut['stem_ht']])[1],
      cg('rectangle', [abut['bw_width'],abut['backw_ht']], [0,abut['stem_ht']+abut['cap']])[1],
      cg('triangle_1', [abut['haunch']['stem_haunch'][0],-abut['haunch']['stem_haunch'][1]], [0,abut['stem_ht']])[1],
      cg('rectangle', [abut['haunch']['approach_slab_haunch_rec'][0],-abut['haunch']['approach_slab_haunch_rec'][1]], [0,abut['total_ht']-abut['approach_slab_dep']])[1],
      cg('triangle_1', [abut['haunch']['approach_slab_haunch_tri'][0],-abut['haunch']['approach_slab_haunch_tri'][1]], [0,abut['total_ht']-abut['approach_slab_dep']-abut['haunch']['approach_slab_haunch_rec'][1]])[1]]
mmt = []
for index, i in enumerate(H_force):
    mmt.append(round(H_force[index]*ht[index],5))
A7 = pd.DataFrame({"Weight":loads, 'Hz force(KN)':H_force,"Lever arm":ht,"Moment":mmt},index = ['Stem(rectangle)','Stem(triangle)','Cap','Backwall','Stem haunch','Approach slab haunch(rec)','Approach slab haunch(tri)'])
# display(A7)
A7.loc[len(A7.index)] = [None, A7['Hz force(KN)'].sum(),None,A7['Moment'].sum()]
A7.loc[len(A7.index)] = [dl, dl*ahh, abut['total_ht']-box['pillar_hei']/3-abut['wc'],(dl*ahh)*(abut['total_ht']-box['pillar_hei']/3-abut['wc'])]
display(A7.rename(index={7:'Total',8:'Superstructure DL'}))

,Weight,Hz force(KN),Lever arm,Moment
Stem(rectangle),12.571875,2.26294,0.050000,0.11315
Stem(triangle),0.506250,0.09112,0.033330,0.00304
Cap,1.687500,0.30375,0.200000,0.06075
Backwall,28.096875,5.05744,1.687500,8.53443
Stem haunch,78.574219,14.14336,-0.733330,-10.37175
Approach slab haunch(rec),2.109375,0.37969,2.750000,1.04415
Approach slab haunch(tri),1.054688,0.18984,2.458330,0.46669
Total,NaN,22.42814,NaN,-0.14954
Superstructure DL,5220.625000,939.71250,2.333333,2192.66250


Due to Earth pressure

In [18]:
C = ca(alpha, beta, delta, phi, lam, av)
k = ka(alpha, beta, delta, phi)
ep = (C-k)*h*unitwt_bfill 
A8 = pd.DataFrame({"Force":.5*ep*h*abut['width_of_abut'],"Lever arm":h/3,"Moment":(.5*ep*h*abut['width_of_abut'])*h/3},index = ['Seismic force due to backfill'])
display(A8)
print(ep)

,Force,Lever arm,Moment
Seismic force due to backfill,22.335041,0.958333,21.404415


11.5092


Dynamic Surcharge 

In [19]:
A9 = pd.DataFrame({"Force":(unitwt_bfill*(C-k)*hl*abut['width_of_abut']*h),"Lever arm":(0.66*abut['total_ht']),"Moment":((unitwt_bfill*(C-k)*hl*abut['width_of_abut']*h)*(0.66*abut['total_ht']))},index = ['Dynamic Surcharge'])
display(A9)
print(hl)
print(C-k)

,Force,Lever arm,Moment
Dynamic Surcharge,18.644904,2.2935,42.762087


1.2
0.20016


Summary of loads

In [20]:
#inputs
#Unfactored Forces and their moments

force_ver = [A1.loc[7,'Load'],A2.iloc[0,0],A2.iloc[1,0],A2.iloc[2,0],0,0,A3.iloc[2,0],0,0,0,0,0,0,0]
force_hz = [0,0,0,0,A3.iloc[0,0],A3.iloc[1,0],0,A6.iloc[0,0],A4.iloc[0,0],A5.iloc[0,0],A7.iloc[0,0],A7.iloc[7,1],A8.iloc[0,0],A9.iloc[0,0]]
moment_unf = [A1.iloc[7,2],A2.iloc[0,2],A2.iloc[1,2],A2.iloc[2,2],A3.iloc[0,2],A3.iloc[1,2],A3.iloc[2,2],A6.iloc[0,2],A4.iloc[0,2],A5.iloc[0,2],A7.iloc[8,3],A7.iloc[7,3],A8.iloc[0,2],A9.iloc[0,2]]

#Safety Factors 

non_ses = [1.35,1.35,1.5,1.75,1.5,0,1,1.2,1.5,0.9,0,0,0,0]
ses = [1.35,1.35,0,1.75,1,0,0,0.2,0.2,0.5,1.5,1.5,1.5,1]

In [21]:
#Factored responses (Non Seismic)

ns_force_ver = []
ns_force_hz = []
ns_moment = []
for index, i in enumerate(force_ver):
    ns_force_ver.append(round(force_ver[index]*non_ses[index],5))
for index, i in enumerate(force_hz):
    ns_force_hz.append(round(force_hz[index]*non_ses[index],5))
for index, i in enumerate(moment_unf):
    ns_moment.append(round(moment_unf[index]*non_ses[index],5))
A10 = pd.DataFrame({"Vertical Forces":ns_force_ver, 'Horizontal forces':ns_force_hz,"Moment":ns_moment},index = ['Abutment self','SS DL','SS Live load','Surface DL','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge','Braking','Temperature','SS DL(seismic)','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
A10.loc[len(A10.index)] = [A10['Vertical Forces'].sum(), A10['Horizontal forces'].sum(),A10['Moment'].sum()]
display(A10.rename(index={14:'Total'}))

#Factored responses (Seismic)

ses_force_ver = []
ses_force_hz = []
ses_moment = []
for index, i in enumerate(force_ver):
    ses_force_ver.append(round(force_ver[index]*ses[index],5))
for index, i in enumerate(force_hz):
    ses_force_hz.append(round(force_hz[index]*ses[index],5))
for index, i in enumerate(moment_unf):
    ses_moment.append(round(moment_unf[index]*ses[index],5))
A11 = pd.DataFrame({"Vertical Forces":ses_force_ver, 'Horizontal forces':ses_force_hz,"Moment":ses_moment},index = ['Abutment self','SS DL','SS Live load','Surface DL','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge','Braking','Temperature','SS DL(seismic)','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
A11.loc[len(A11.index)] = [A11['Vertical Forces'].sum(), A11['Horizontal forces'].sum(),A11['Moment'].sum()]
display(A11.rename(index={14:'Total'}))


,Vertical Forces,Horizontal forces,Moment
Abutment self,168.21105,0.00000,-257.79417
SS DL,7047.84375,0.00000,-22376.90391
SS Live load,1343.28600,0.00000,-4264.93305
Surface DL,375.37500,0.00000,-1191.81562
Earth Hz (dry),0.00000,41.04182,49.55799
Earth Hz (wet),0.00000,0.00000,0.00000
Earth ver,11.05465,0.00000,-22.24748
Live load Surcharge,0.00000,29.56134,34.24188
Braking,0.00000,332.40000,1553.97000
Temperature,0.00000,288.17875,172.90725


,Vertical Forces,Horizontal forces,Moment
Abutment self,168.21105,0.00000,-257.79417
SS DL,7047.84375,0.00000,-22376.90391
SS Live load,0.00000,0.00000,-0.00000
Surface DL,375.37500,0.00000,-1191.81562
Earth Hz (dry),0.00000,27.36121,33.03866
Earth Hz (wet),0.00000,0.00000,0.00000
Earth ver,0.00000,0.00000,-0.00000
Live load Surcharge,0.00000,4.92689,5.70698
Braking,0.00000,44.32000,207.19600
Temperature,0.00000,160.09931,96.05958


**Foundation Stability**

Load from self weight of abutment

In [22]:
la_fromtoe = [cg('rectangle',[abut['stem_width'],abut['stem_ht']],[fdn['toe']+abut['be_taperwidth'],0])[0],
             cg('triangle_2',[abut['be_taperwidth'],abut['stem_ht']],[fdn['toe'],0])[0],
             cg('rectangle',[abut['cap_width'],abut['cap']],[fdn['toe']+abut['be_taperwidth'],abut['stem_ht']])[0],
             cg('rectangle',[abut['bw_width'],abut['backw_ht']],[fdn['toe']+abut['ttl_stembase_width']+abut['haunch']['stem_haunch'][0]-abut['bw_width'],abut['stem_ht']+abut['cap']])[0],
             cg('triangle_1',[abut['haunch']['stem_haunch'][0],abut['haunch']['stem_haunch'][1]],[fdn['toe']+abut['ttl_stembase_width'],abut['stem_ht']-abut['haunch']['stem_haunch'][1]])[0],
             cg('rectangle',[abut['haunch']['approach_slab_haunch_rec'][0],abut['haunch']['approach_slab_haunch_rec'][1]],[fdn['toe']+abut['ttl_stembase_width'],abut['stem_ht']+abut['cap']+abut['backw_ht']-abut['stem_ht']-abut['haunch']['approach_slab_haunch_rec'][1]])[0],
             cg('triangle_1',[abut['haunch']['approach_slab_haunch_tri'][0],abut['haunch']['approach_slab_haunch_tri'][1]],[fdn['toe']+abut['ttl_stembase_width'],abut['stem_ht']+abut['cap']+abut['backw_ht']-abut['stem_ht']-abut['haunch']['approach_slab_haunch_rec'][1]])[0]]
mom_abttoe = []
for index,i in enumerate(loads):
    mom_abttoe.append(loads[index]*la_fromtoe[index])
B1 = pd.DataFrame({"Load":loads,"Lever arm":la_fromtoe,"Moment":mom_abttoe},index = ['Stem(rectangle)','Stem(triangle)','Cap','Backwall','Stem haunch','Approach slab haunch(rec)','Approach slab haunch(tri)'])
B1.loc[len(B1.index)] = [B1['Load'].sum(),None,B1['Moment'].sum()]
B1 = B1.rename(index={7:'Total'})
B1.loc[len(B1.index)] = [(unitwt_conc*fdn['len']*fdn['th']*fdn['wid']),fdn['len']/2,(unitwt_conc*fdn['len']*fdn['th']*fdn['wid'])*fdn['len']/2]
display(B1.rename(index={8:'Foundation Pad'}))


,Load,Lever arm,Moment
Stem(rectangle),12.571875,10.56250,132.790430
Stem(triangle),0.506250,8.50000,4.303125
Cap,1.687500,8.82500,14.892187
Backwall,28.096875,11.97500,336.460078
Stem haunch,78.574219,10.88333,855.149152
Approach slab haunch(rec),2.109375,8.52500,17.982422
Approach slab haunch(tri),1.054688,8.56667,9.035160
Total,124.600781,NaN,1370.612554
Foundation Pad,1635.000000,5.45000,8910.750000


Load from superstructure

In [23]:
arm_toe = [(fdn['len']-fdn['heel']+abut['haunch']['stem_haunch'][0]-abut['bw_width']-abut['bw_to_bearing_c']),
          (fdn['len']-fdn['heel']+abut['haunch']['stem_haunch'][0]-abut['bw_width']-abut['bw_to_bearing_c']),
          (fdn['len']-fdn['heel']+abut['haunch']['stem_haunch'][0]-abut['bw_width']-abut['bw_to_bearing_c'])]
momentabt_toe = []
for index,i in enumerate(load):
    momentabt_toe.append(load[index]*arm_toe[index])
B2 = pd.DataFrame({"Load":load,"Lever arm":arm_toe,"Moment":momentabt_toe},index = ['Dead load','Live load','Surface Load'])
display(B2)

,Load,Lever arm,Moment
Dead load,5220.625,11.575,60428.734375
Live load,895.524,11.575,10365.690300
Surface Load,214.500,11.575,2482.837500


Earth Pressure

In [24]:
ht1 = h + fdn['th']
ep_new = unitwt_bfill*ht1*ka(alpha, beta, delta, phi) #Earth pressure in KN/m^2.
force_new = 0.5*ep_new*(h+fdn['th'])*fdn['wid'] #Force due to earth pressure in KN.
forc_new = [force_new*math.cos(math.radians(delta)),force_new*math.cos(math.radians(delta)),force_new*math.sin(math.radians(delta))] #Horizontal and vertical components.
larm_abttoe = [-0.42*ht1, -0.33*ht1, (fdn['len']-fdn['heel']) ] #-ve for overturning moments.
print(force_new)
momen_abt_toe = []
for index,i in enumerate(forc_new):
    momen_abt_toe.append(forc_new[index]*larm_abttoe[index])
B3 = pd.DataFrame({'Force(KN)':forc_new, 'Lever arm(m)':larm_abttoe,'Moment(KNm)':momen_abt_toe}, index = ['Earth pressure Hz(dry)','Earth pressure Hz(wet)','Earth pressure Ver'])
display(B3)

187.49800781249996


,Force(KN),Lever arm(m),Moment(KNm)
Earth pressure Hz(dry),173.845126,-2.88750,-501.977800
Earth pressure Hz(wet),173.845126,-2.26875,-394.411129
Earth pressure Ver,70.237990,8.40000,589.999116


Braking

In [25]:
br_forces = [max(F),-1*(max(F)*pt/box['span'])]
la_toe = [-(ht1+abut['approach_slab_dep']+pt),(fdn['len']-fdn['heel']+abut['haunch']['stem_haunch'][0]-abut['bw_width']-abut['bw_to_bearing_c'])]
m_toe = []
for index,i in enumerate(br_forces):
    m_toe.append(br_forces[index]*la_toe[index])
B4 = pd.DataFrame({'Force(KN)':br_forces, 'Lever arm(m)':la_toe,'Moment(KNm)':m_toe}, index = ['Braking load','Braking vertical reaction'])
display(B4)

,Force(KN),Lever arm(m),Moment(KNm)
Braking load,221.6000,-8.675,-1922.38000
Braking vertical reaction,-5.3184,11.575,-61.56048


Temperature Forces

In [26]:
La_toe = -(abut['stem_ht']+abut['cap']+bearing_ht+fdn['th'])
B5 = pd.DataFrame({'Force(KN)':frc, 'Lever arm(m)':La_toe,'Moment(KNm)': (frc*La_toe)}, index = ['Temperature force'])
display(B5)

,Force(KN),Lever arm(m),Moment(KNm)
Temperature force,320.198612,-4.6,-1472.913614


Live Load Surcharge

In [27]:
B6 = pd.DataFrame({'Force(KN)':(Pr*abut['width_of_abut']*abut['total_ht']), 'Lever arm(m)':-(abut['total_ht']+fdn['th'])/2,'Moment(KNm)':-((Pr*abut['width_of_abut']*abut['total_ht'])*(abut['total_ht']+fdn['th'])/2)}, index = ['Force due to LL surcharge'])
B6.loc[len(B6.index)] = [(unitwt_bfill*pt*abut['width_of_abut']*fdn['heel']), (fdn['heel']/2+fdn['toe']+abut['ttl_stembase_width']),(unitwt_bfill*pt*abut['width_of_abut']*fdn['heel'])*(fdn['heel']/2+fdn['toe']+abut['ttl_stembase_width'])]
display(B6.rename(index={1:'Vertical Surcharge'}))

,Force(KN),Lever arm(m),Moment(KNm)
Force due to LL surcharge,29.775551,-3.7375,-111.286123
Vertical Surcharge,81.000000,9.6500,781.650000


Backfill

In [28]:
fbfill = unitwt_bfill*fdn['heel']*fdn['wid']*h
la_filltoe = fdn['len']-fdn['heel']/2
B7 = pd.DataFrame({'Force(KN)':fbfill, 'Lever arm(m)':la_filltoe,'Moment(KNm)':fbfill*la_filltoe}, index = ['Backfill load'])
display(B7)

,Force(KN),Lever arm(m),Moment(KNm)
Backfill load,215.625,9.65,2080.78125


Seismic Forces

In [29]:
la_abutses = [-cg('rectangle',[abut['stem_width'],abut['stem_ht']],[fdn['toe']+abut['be_taperwidth'],fdn['th']])[1],
             -cg('triangle_2',[abut['be_taperwidth'],abut['stem_ht']],[fdn['toe'],fdn['th']])[1],
             -cg('rectangle',[abut['cap_width'],abut['cap']],[fdn['toe']+abut['be_taperwidth'],abut['stem_ht']+fdn['th']])[1],
             -cg('rectangle',[abut['bw_width'],abut['backw_ht']],[fdn['toe']+abut['ttl_stembase_width']+abut['haunch']['stem_haunch'][0],abut['stem_ht']+abut['cap']+fdn['th']])[1],
             -cg('triangle_1',[abut['haunch']['stem_haunch'][0],abut['haunch']['stem_haunch'][1]],[fdn['toe']+abut['ttl_stembase_width'],abut['stem_ht']-abut['haunch']['stem_haunch'][1]+fdn['th']])[1],
             -cg('rectangle',[abut['haunch']['approach_slab_haunch_rec'][0],abut['haunch']['approach_slab_haunch_rec'][1]],[fdn['toe']+abut['cap_width'],abut['stem_ht']+abut['cap']+abut['backw_ht']-abut['haunch']['approach_slab_haunch_rec'][1]+fdn['th']])[1],
             -cg('triangle_1',[abut['haunch']['approach_slab_haunch_tri'][0],abut['haunch']['approach_slab_haunch_tri'][1]],[fdn['toe']+abut['cap_width'],abut['stem_ht']+abut['cap']+abut['backw_ht']-abut['haunch']['approach_slab_haunch_rec'][1]+fdn['th']])[1]]
mmt_toeses = []
for index, i in enumerate(H_force):
    mmt_toeses.append(round(H_force[index]*la_abutses[index],5))
B8 = pd.DataFrame({"Weight":loads, 'Hz force(KN)':H_force,"Lever arm":la_abutses,"Moment":mmt_toeses},index = ['Stem(rectangle)','Stem(triangle)','Cap','Backwall','Stem haunch','Approach slab haunch(rec)','Approach slab haunch(tri)'])
# display(A7)
B8.loc[len(B8.index)] = [None, B8['Hz force(KN)'].sum(),None,B8['Moment'].sum()]
B8 = B8.rename(index={7:'Total'})
B8.loc[len(B8.index)] = [B1.iloc[8,0],B1.iloc[8,0]*ahh,-fdn['th']/2,-B1.iloc[8,0]*ahh*fdn['th']/2]
B8 = B8.rename(index={8:'Foundation Pad'})
display(B8)

,Weight,Hz force(KN),Lever arm,Moment
Stem(rectangle),12.571875,2.26294,-4.05000,-9.16491
Stem(triangle),0.506250,0.09112,-4.03333,-0.36752
Cap,1.687500,0.30375,-4.20000,-1.27575
Backwall,28.096875,5.05744,-5.68750,-28.76419
Stem haunch,78.574219,14.14336,-3.68333,-52.09466
Approach slab haunch(rec),2.109375,0.37969,-6.95000,-2.63885
Approach slab haunch(tri),1.054688,0.18984,-6.99167,-1.32730
Total,NaN,22.42814,NaN,-95.63318
Foundation Pad,1635.000000,294.30000,-2.00000,-588.60000


Superstructure DL

In [30]:
B9 = pd.DataFrame({"Weight":dl, "Hz Force(KN)":dl*ahh, "Lever arm":-(abut['total_ht']-box['pillar_hei']/3-abut['wc']+fdn['th']),"Moment":-(dl*ahh)*(abut['total_ht']-box['pillar_hei']/3-abut['wc']+fdn['th'])}, index = ['Superstructure DL'])
display(B9)

,Weight,Hz Force(KN),Lever arm,Moment
Superstructure DL,5220.625,939.7125,-6.333333,-5951.5125


Earth Pressure

In [31]:
ep_fdn = (C-k)*ht1*unitwt_bfill
forc_ep = .5*ep_fdn*ht1*fdn['wid']
la_eptoe = -0.66*ht1
B10 = pd.DataFrame({"Force(KN)":forc_ep, "Lever arm":la_eptoe,"Moment":forc_ep*la_eptoe}, index = ['Seismic force due to backfill'])
display(B10)

,Force(KN),Lever arm,Moment
Seismic force due to backfill,141.910313,-4.5375,-643.918043


Dynamic Surcharge

In [32]:
B11 = pd.DataFrame({"Force":(unitwt_bfill*(C-k)*hl*fdn['wid']*abut['total_ht']),"Lever arm":la_eptoe,"Moment":((unitwt_bfill*(C-k)*hl*fdn['wid']*abut['total_ht'])*la_eptoe)},index = ['Dynamic Surcharge'])
display(B11)

,Force,Lever arm,Moment
Dynamic Surcharge,25.040016,-4.5375,-113.619073


In [33]:
#inputs
#Unfactored Forces and their moments

ver_force = [B1.iloc[7,0],B2.iloc[0,0],B2.iloc[1,0],B2.iloc[2,0],B1.iloc[8,0],0,0,B3.iloc[2,0],0,B6.iloc[1,0],0,0,B7.iloc[0,0],B4.iloc[1,0],0,0,0,0,0]
h_forc = [0,0,0,0,0,0,B3.iloc[1,0],0,B6.iloc[0,0],0,B4.iloc[0,0],B5.iloc[0,0],0,0,B8.iloc[7,1],B8.iloc[8,1],B9.iloc[0,1],B10.iloc[0,0],B11.iloc[0,0]]
unf_moment_res = [B1.iloc[7,2],B2.iloc[0,2],B2.iloc[1,2],B2.iloc[2,2],B1.iloc[8,2],0,0,B3.iloc[2,2],0,B6.iloc[1,2],0,0,B7.iloc[0,2],0,0,0,0,0,0]
unf_moment_over = [0,0,0,0,0,0,B3.iloc[1,2],0,B6.iloc[0,2],0,B4.iloc[0,2],B5.iloc[0,2],0,B4.iloc[1,2],B8.iloc[7,3],B8.iloc[8,3],B9.iloc[0,3],B10.iloc[0,2],B11.iloc[0,2]]
fr_coeff = 0.65
print(ver_force)
##Safety Factors (Basic)

bas_R = [0.9,0.9,0,1,0.9,1,0,1,0,0,0,0,0.9,0,0,0,0,0,0]
bas_O = [1.1,1.1,1.5,1.35,1.1,1.5,0,1.5,1.2,1.2,1.15,1.5,1.1,1.15,0,0,0,0,0]

##Safety Factors (Seismic)

ses_R = [0.9,0.9,0,1,0.9,0,1,1,0,0,0,0,0.9,0,0,0,0,0,0]
ses_O = [1.1,1.1,0,1.35,1.1,0,1,1,0,0,0.2,0,1.1,0.2,1.5,1.5,1.5,1.5,1.5]

[124.60078125000001, 5220.625000000001, 895.524, 214.5, 1635.0, 0, 0, 70.23798997891066, 0, 81.00000000000001, 0, 0, 215.625, -5.3184000000000005, 0, 0, 0, 0, 0]


In [34]:
#Factored Responses (Basic)
bas_force_ver = []
bas_force_hz = []
bas_momentR = []
bas_momentO = []
for index, i in enumerate(ver_force):
     bas_force_ver.append(round(ver_force[index]*bas_R[index],5))
     bas_momentR.append(round(unf_moment_res[index]*bas_R[index],5))
bas_force_ver[13] = ver_force[13]*bas_O[13]
for index, i in enumerate(h_forc):
    bas_force_hz.append(round(h_forc[index]*bas_O[index],5))
    bas_momentO.append(round(unf_moment_over[index]*bas_O[index],5))
B12 = pd.DataFrame({"Vertical Forces":bas_force_ver, 'Horizontal forces':bas_force_hz,'Resisting Moment':bas_momentR,'Overturning moment':bas_momentO},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge(HZ)','Live load surcharge(Ver)','Braking','Temperature','Backfill','Braking uplift','SS DL(seismic)','Seismic Foundation','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
B12.loc[len(B12.index)] = [B12['Vertical Forces'].sum(), B12['Horizontal forces'].sum(),B12['Resisting Moment'].sum(),B12['Overturning moment'].sum()]
display(B12.rename(index={19:'Total'}))

#Factored Responses (Seismic)
ses_force_ver = []
ses_force_hz = []
ses_momentR = []
ses_momentO = []
for index, i in enumerate(ver_force):
    ses_force_ver.append(round(ver_force[index]*ses_R[index],5))
    ses_momentR.append(round(unf_moment_res[index]*ses_R[index],5))
ses_force_ver[13] = ver_force[13]*ses_O[13]
for index, i in enumerate(h_forc):
    ses_force_hz.append(round(h_forc[index]*ses_O[index],5))
    ses_momentO.append(round(unf_moment_over[index]*ses_O[index],5))
B13 = pd.DataFrame({"Vertical Forces":ses_force_ver, 'Horizontal forces':ses_force_hz,'Resisting Moment':ses_momentR,'Overturning moment':ses_momentO},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge(HZ)','Live load surcharge(Ver)','Braking','Temperature','Backfill','Braking uplift','SS DL(seismic)','Seismic Foundation','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
B13.loc[len(B13.index)] = [B13['Vertical Forces'].sum(), B13['Horizontal forces'].sum(),B13['Resisting Moment'].sum(),B13['Overturning moment'].sum()]
# display(B13.rename(index={19:'Total'}))

#Overturning and sliding check
fact_basic = [B12.iloc[19,2]/-(B12.iloc[19,3]), (fr_coeff*B12.iloc[19,0])/B12.iloc[19,1]]
fact_ses = [B13.iloc[19,2]/-(B13.iloc[19,3]), (fr_coeff*B13.iloc[19,0])/B13.iloc[19,1]]
F = pd.DataFrame({"Basic":fact_basic, "Seismic":fact_ses}, index = ['Overturning','Sliding'])
# display(F)

,Vertical Forces,Horizontal forces,Resisting Moment,Overturning moment
Abutment self,112.14070,0.00000,1233.55130,0.00000
SS DL,4698.56250,0.00000,54385.86094,0.00000
SS Live load,0.00000,0.00000,0.00000,0.00000
Surface DL,214.50000,0.00000,2482.83750,0.00000
Foundation,1471.50000,0.00000,8019.67500,0.00000
Earth Hz (dry),0.00000,0.00000,0.00000,0.00000
Earth Hz (wet),0.00000,0.00000,0.00000,-0.00000
Earth ver,70.23799,0.00000,589.99912,0.00000
Live load Surcharge(HZ),0.00000,35.73066,0.00000,-133.54335
Live load surcharge(Ver),0.00000,0.00000,0.00000,0.00000


Bearing Pressure foundation

In [35]:
#Inputs
ex = -fdn['len']/2
ey = 0
c1 = [1,1,1,1,1,1,0,1,1,1,1,1,1,1,0,0,0,0,0]
c2 = [1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1]
c3 = [1,1,0,1,1,0,1,1,0,0,0,1,1,0,1,1,1,1,0]

In [36]:
print(len(unf_moment_over))
tot_unf_M = []
for i,j in zip(unf_moment_res,unf_moment_over):
    if i != 0:
        tot_unf_M.append(i)
    elif j!=0:
        tot_unf_M.append(j)
tot_unf_M.insert(5,0)
# print(tot_unf_M)

C1 = pd.DataFrame({"Vertical Forces":ver_force, 'Horizontal forces':h_forc,'Moment':tot_unf_M},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load surcharge (Hz)','Live load surcharge(Ver)','Braking','Temperature','Backfill','Braking uplift','SS DL(seismic)','Seismic Foundation','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
C1.loc[len(C1.index)] = [C1['Vertical Forces'].sum(), C1['Horizontal forces'].sum(),C1['Moment'].sum()]
display(C1.rename(index={19:'Total'}))



19


,Vertical Forces,Horizontal forces,Moment
Abutment self,124.600781,0.000000,1370.612554
SS DL,5220.625000,0.000000,60428.734375
SS Live load,895.524000,0.000000,10365.690300
Surface DL,214.500000,0.000000,2482.837500
Foundation,1635.000000,0.000000,8910.750000
Earth Hz (dry),0.000000,0.000000,0.000000
Earth Hz (wet),0.000000,173.845126,-394.411129
Earth ver,70.237990,0.000000,589.999116
Live load surcharge (Hz),0.000000,29.775551,-111.286123
Live load surcharge(Ver),81.000000,0.000000,781.650000


In [37]:
new_M = []
for i,j,k in zip(tot_unf_M,ver_force,h_forc):
    new_M.append(i+j*ex+k*ey)
print(new_M)
C2 = pd.DataFrame({"Vertical Forces":ver_force, 'Horizontal forces':h_forc,'Moment':new_M},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load surcharge (Hz)','Live load surcharge(Ver)','Braking','Temperature','Backfill','Braking uplift','SS DL(seismic)','Seismic Foundation','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
display(C2)


[691.5382962890626, 31976.328125, 5485.0845, 1313.8124999999995, 0.0, 0.0, -394.4111287213063, 207.20207043778646, -111.28612335750002, 340.2000000000001, -1922.3799999999999, -1472.9136141176473, 905.625, -32.575199999999995, -95.63318, -588.6, -5951.512500000001, -643.9180429687501, -113.61907260000001]


,Vertical Forces,Horizontal forces,Moment
Abutment self,124.600781,0.000000,691.538296
SS DL,5220.625000,0.000000,31976.328125
SS Live load,895.524000,0.000000,5485.084500
Surface DL,214.500000,0.000000,1313.812500
Foundation,1635.000000,0.000000,0.000000
Earth Hz (dry),0.000000,0.000000,0.000000
Earth Hz (wet),0.000000,173.845126,-394.411129
Earth ver,70.237990,0.000000,207.202070
Live load surcharge (Hz),0.000000,29.775551,-111.286123
Live load surcharge(Ver),81.000000,0.000000,340.200000


In [38]:
# combination 1
c1_verf = []
c1_hzf = []
c1_mom = []
for index, i in enumerate(c1):
    c1_verf.append(round(c1[index]*ver_force[index],5))
    c1_hzf.append(round(c1[index]*h_forc[index],5))
    c1_mom.append(round(c1[index]*new_M[index],5))
C3 = pd.DataFrame({"Vertical Forces":c1_verf, 'Horizontal forces':c1_hzf,'Moment':c1_mom},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load surcharge (Hz)','Live load surcharge(Ver)','Braking','Temperature','Backfill','Braking uplift','SS DL(seismic)','Seismic Foundation','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
C3.loc[len(C3.index)] = [C3['Vertical Forces'].sum(), C3['Horizontal forces'].sum(),C3['Moment'].sum()]
display(C3.rename(index={19:'Total'}))

#Combination 2
c2_verf = []
c2_hzf = []
c2_mom = []
for index, i in enumerate(c1):
    c2_verf.append(round(c2[index]*ver_force[index],5))
    c2_hzf.append(round(c2[index]*h_forc[index],5))
    c2_mom.append(round(c2[index]*new_M[index],5))
C4 = pd.DataFrame({"Vertical Forces":c2_verf, 'Horizontal forces':c2_hzf,'Moment':c2_mom},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load surcharge (Hz)','Live load surcharge(Ver)','Braking','Temperature','Backfill','Braking uplift','SS DL(seismic)','Seismic Foundation','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
C4.loc[len(C4.index)] = [C4['Vertical Forces'].sum(), C4['Horizontal forces'].sum(),C4['Moment'].sum()]
display(C4.rename(index={19:'Total'}))

#Combination 3
c3_verf = []
c3_hzf = []
c3_mom = []
for index, i in enumerate(c1):
    c3_verf.append(round(c3[index]*ver_force[index],5))
    c3_hzf.append(round(c3[index]*h_forc[index],5))
    c3_mom.append(round(c3[index]*new_M[index],5))
C5 = pd.DataFrame({"Vertical Forces":c3_verf, 'Horizontal forces':c3_hzf,'Moment':c3_mom},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load surcharge (Hz)','Live load surcharge(Ver)','Braking','Temperature','Backfill','Braking uplift','SS DL(seismic)','Seismic Foundation','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
C5.loc[len(C5.index)] = [C5['Vertical Forces'].sum(), C5['Horizontal forces'].sum(),C5['Moment'].sum()]
display(C5.rename(index={19:'Total'}))


,Vertical Forces,Horizontal forces,Moment
Abutment self,124.60078,0.00000,691.53830
SS DL,5220.62500,0.00000,31976.32812
SS Live load,895.52400,0.00000,5485.08450
Surface DL,214.50000,0.00000,1313.81250
Foundation,1635.00000,0.00000,0.00000
Earth Hz (dry),0.00000,0.00000,0.00000
Earth Hz (wet),0.00000,0.00000,-0.00000
Earth ver,70.23799,0.00000,207.20207
Live load surcharge (Hz),0.00000,29.77555,-111.28612
Live load surcharge(Ver),81.00000,0.00000,340.20000


,Vertical Forces,Horizontal forces,Moment
Abutment self,124.60078,0.00000,691.53830
SS DL,5220.62500,0.00000,31976.32812
SS Live load,895.52400,0.00000,5485.08450
Surface DL,214.50000,0.00000,1313.81250
Foundation,1635.00000,0.00000,0.00000
Earth Hz (dry),0.00000,0.00000,0.00000
Earth Hz (wet),0.00000,173.84513,-394.41113
Earth ver,70.23799,0.00000,207.20207
Live load surcharge (Hz),0.00000,29.77555,-111.28612
Live load surcharge(Ver),81.00000,0.00000,340.20000


,Vertical Forces,Horizontal forces,Moment
Abutment self,124.60078,0.00000,691.53830
SS DL,5220.62500,0.00000,31976.32812
SS Live load,0.00000,0.00000,0.00000
Surface DL,214.50000,0.00000,1313.81250
Foundation,1635.00000,0.00000,0.00000
Earth Hz (dry),0.00000,0.00000,0.00000
Earth Hz (wet),0.00000,173.84513,-394.41113
Earth ver,70.23799,0.00000,207.20207
Live load surcharge (Hz),0.00000,0.00000,-0.00000
Live load surcharge(Ver),0.00000,0.00000,0.00000


***Design of abutment stem***

In [59]:
#INPUTS
# Ecu = 0.0035 #Limiting strain in concrete.
# Es = 0.87*fy/E
# ce = 75 #Effective cover in mm.
# d = ((abut['ttl_stembase_width']*1000) - ce - 10) #Effective depth of beam.
# beta1 = 0.80952
# beta2 = 0.416
# fyd = fy/1.15
# fctm = 0.259*pow(fck,2/3)
# dia = 20 #Diameter of reinforcement bars in mm.


In [61]:
axf_c = 0.446*fck*abut['ttl_stembase_width']*abut['width_of_abut']*0.1*1000
print(abut['width_of_abut'])
res = [A10.iloc[14,0],A11.iloc[14,0]]
print(axf_c)
des_mom = [A10.iloc[14,2],A11.iloc[14,2]]
if axf_c > max(res):
    print("The member is designed for pure flexure.")

1.35
0.0


Calculation of limiting values

In [41]:
#Inputs

Ecu = 0.0035 #Limiting strain in concrete.
Es = 0.87*fy/E
ce = 75 #Effective cover in mm.
d = ((abut['ttl_stembase_width']*1000) - ce - 10) #Effective depth of beam.
beta1 = 0.80952
beta2 = 0.416
fyd = fy/1.15
fctm = 0.259*pow(fck,2/3)
dia = 20 #Diameter of reinforcement bars in mm.
print(d)

-85.0


In [42]:
xu_lim = (Ecu*d)/(Ecu+Es) #Limiting depth of neutral axis in mm
cg_steel = d-0.416*xu_lim #CG from level of steel in mm.
comp_force = (0.446*fck*beta1*(1000*abut['width_of_abut'])*xu_lim)/1000 
mu_lim = (comp_force*cg_steel)/1000
print(mu_lim)

56.512945529208665


Calculation of Moment of resistance of section

In [43]:
xu = int(input("Enter a trial value of depth of NA:"))
cgs = d-0.416*xu
compf = (0.446*fck*beta1*(1000*abut['width_of_abut'])*xu)/1000 
mor = (cgs*compf)/1000
x = na(xu,cgs,compf,max(des_mom))
# print(x)

Enter a trial value of depth of NA:150


Calculation of steel required

In [44]:
steel_req = (compf*1000)/fyd #Area of reinforcements required in mm^2.

#Minimum steel required
ast_minr = round(0.26*fctm/fy,5) if 0.26*fctm/fy>0.0013 else 0.0013
ast_min = ast_minr*abut['width_of_abut']*d
Check1 = "OK" if steel_req>=ast_min else "Not OK"
if Check1 == "Not OK":
    x = na(xu,cgs,compf,max(des_mom))

#Maximum steel required
ast_max = 0.025*(abut['width_of_abut']*abut['ttl_stembase_width'])*pow(10,6)
Check2 = "OK" if steel_req<=ast_max else "Not OK"
if Check2 == "Not OK":
    x = na(xu,cgs,compf,max(des_mom))
print(Check2)
print(steel_req)

Not OK
5885.4998034


In [45]:
area_bar = math.pi * pow(dia,2)/4
spacing = 120
ast_pro = area_bar*abut['width_of_abut']*1000/spacing
while ast_pro<=steel_req:
    spacing = int(input("Enter lower value of spacing:"))
    ast_pro = area_bar*abut['width_of_abut']*1000/spacing
print(spacing)
print("OK")

Enter lower value of spacing:70
70
OK


Shear Check

In [46]:
v_min = min(res)*1000
ac = abut['width_of_abut']*1000*d
sigma_c = v_min/ac
k = 1+ pow(200/d,.5) if (1+ pow(200/d,.5))<=2 else 2
row_1 = ast_pro/(abut['width_of_abut']*1000*d)
print(row_1)
t_min = 0.031*pow(k,1.5)*pow(fck,0.5)
vrdc_min = (t_min + 0.15*sigma_c)*abut['width_of_abut']*1000*d/1000
if (0.12*k*pow(80*row_1*fck,0.33)+0.15*sigma_c)*abut['width_of_abut']*1000*d/1000>=vrdc_min:
    vrdc = (0.12*k*pow(80*row_1*fck,0.33)+0.15*sigma_c)*abut['width_of_abut']*1000*d/1000
else:
    vrdc = vrdc_min
    
#Check
shear_f = max([A10.iloc[14,1],A11.iloc[14,1]])
print(vrdc,shear_f)
if vrdc >= shear_f:
    print("OK")
else:
    print("Not OK")

-0.0527998765309209
1079.1902924122332 691.18191
OK


C:\Users\MILANJ~1\AppData\Local\Temp/ipykernel_11652/192765419.py:4: RuntimeWarning: invalid value encountered in double_scalars
  k = 1+ pow(200/d,.5) if (1+ pow(200/d,.5))<=2 else 2
C:\Users\MILANJ~1\AppData\Local\Temp/ipykernel_11652/192765419.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if (0.12*k*pow(80*row_1*fck,0.33)+0.15*sigma_c)*abut['width_of_abut']*1000*d/1000>=vrdc_min:


Calculation of Crack Width

In [47]:
#Inputs
hc_eff = min([2.5*(abut['ttl_stembase_width']-d), (abut['ttl_stembase_width']-xu)/3, abut['ttl_stembase_width']/2])
ac_eff = abut['width_of_abut']*hc_eff
k_t = 0.5
f_cm = fck+10
c = 75
#Factors
rare = [1,1,1,1.2,1,0,1,0.8,0.75,0.6,0,0,0,0]
freq = [1,1,0.75,1.2,1,0,1,0,0.2,0.5,0,0,0,0]
quasi = [1,1,0,1.2,1,0,1,0,0,0.5,0,0,0,0]

# Rare combination
v_rare = []
hz_rare = []
mom_rare = []
for index,i in enumerate(rare):
    v_rare.append(round(rare[index]*force_ver[index],5))
    hz_rare.append(round(rare[index]*force_hz[index],5))
    mom_rare.append(round(rare[index]*moment_unf[index],5))
W1 = pd.DataFrame({"Vertical Forces":v_rare, 'Horizontal forces':hz_rare,"Moment":mom_rare},index = ['Abutment self','SS DL','SS Live load','Surface DL','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge','Braking','Temperature','SS DL(seismic)','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
W1.loc[len(W1.index)] = [W1['Vertical Forces'].sum(), W1['Horizontal forces'].sum(), W1['Moment'].sum()]
display(W1.rename(index={14:'Total'}))

# Frequent combination
v_freq = []
hz_freq = []
mom_freq = []
for index,i in enumerate(freq):
    v_freq.append(round(freq[index]*force_ver[index],5))
    hz_freq.append(round(freq[index]*force_hz[index],5))
    mom_freq.append(round(freq[index]*moment_unf[index],5))
W2 = pd.DataFrame({"Vertical Forces":v_freq, 'Horizontal forces':hz_freq,"Moment":mom_freq},index = ['Abutment self','SS DL','SS Live load','Surface DL','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge','Braking','Temperature','SS DL(seismic)','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
W2.loc[len(W2.index)] = [W2['Vertical Forces'].sum(), W2['Horizontal forces'].sum(), W2['Moment'].sum()]
display(W2.rename(index={14:'Total'}))

# Quasi-permanent combination
v_quasi = []
hz_quasi = []
mom_quasi = []
for index,i in enumerate(quasi):
    v_quasi.append(round(quasi[index]*force_ver[index],5))
    hz_quasi.append(round(quasi[index]*force_hz[index],5))
    mom_quasi.append(round(quasi[index]*moment_unf[index],5))
W3 = pd.DataFrame({"Vertical Forces":v_quasi, 'Horizontal forces':hz_quasi,"Moment":mom_quasi},index = ['Abutment self','SS DL','SS Live load','Surface DL','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge','Braking','Temperature','SS DL(seismic)','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
W3.loc[len(W3.index)] = [W3['Vertical Forces'].sum(), W3['Horizontal forces'].sum(), W3['Moment'].sum()]
display(W3.rename(index={14:'Total'}))

,Vertical Forces,Horizontal forces,Moment
Abutment self,124.60078,0.00000,-190.95864
SS DL,5220.62500,0.00000,-16575.48438
SS Live load,895.52400,0.00000,-2843.28870
Surface DL,257.40000,0.00000,-817.24500
Earth Hz (dry),0.00000,27.36121,33.03866
Earth Hz (wet),0.00000,0.00000,0.00000
Earth ver,11.05465,0.00000,-22.24748
Live load Surcharge,0.00000,19.70756,22.82792
Braking,0.00000,166.20000,776.98500
Temperature,0.00000,192.11917,115.27150


,Vertical Forces,Horizontal forces,Moment
Abutment self,124.60078,0.00000,-190.95864
SS DL,5220.62500,0.00000,-16575.48438
SS Live load,671.64300,0.00000,-2132.46652
Surface DL,257.40000,0.00000,-817.24500
Earth Hz (dry),0.00000,27.36121,33.03866
Earth Hz (wet),0.00000,0.00000,0.00000
Earth ver,11.05465,0.00000,-22.24748
Live load Surcharge,0.00000,0.00000,0.00000
Braking,0.00000,44.32000,207.19600
Temperature,0.00000,160.09931,96.05958


,Vertical Forces,Horizontal forces,Moment
Abutment self,124.60078,0.00000,-190.95864
SS DL,5220.62500,0.00000,-16575.48438
SS Live load,0.00000,0.00000,-0.00000
Surface DL,257.40000,0.00000,-817.24500
Earth Hz (dry),0.00000,27.36121,33.03866
Earth Hz (wet),0.00000,0.00000,0.00000
Earth ver,11.05465,0.00000,-22.24748
Live load Surcharge,0.00000,0.00000,0.00000
Braking,0.00000,0.00000,0.00000
Temperature,0.00000,160.09931,96.05958


In [48]:
bm_crack = min([W1.iloc[14,2],W2.iloc[14,2],W3.iloc[14,2]])
stress_act = (bm_crack*pow(10,6))/(ast_pro*cgs)
fct_eff = 0.259*pow(fck,2/3)
e_cm = 22*pow(f_cm/12.5,0.3)*1000
ro1_eff = ast_pro/ac_eff
alp_e = E/e_cm
if (stress_act-(k_t*fct_eff*(1+alp_e*ro1_eff))/ro1_eff)/E >= 0.6*stress_act/E:
    diff_e = (stress_act-(k_t*fct_eff*(1+alp_e*ro1_eff))/ro1_eff)/E
else:
    diff_e = 0.6*stress_act/E
sr_max = 3.4*c+(0.17*dia)/ro1_eff

#Crack width in mm
cr_wid = sr_max*diff_e
#Check
print("OK") if cr_wid <= 0.3 else print("Not OK")

Not OK


Horizontal Reinforcements

In [49]:
check_ar = math.pi*(pow(dia,2)/4)*(1000/spacing)
print(check_ar)
ar_list = [0.25*check_ar, (0.001*1000*abut['ttl_stembase_width']*1000)/2]
spacing_h = int(input("Enter trial spacing:"))
dia_h = int(input("Enter a trial diameter:"))
while (math.pi*pow(dia_h,2)/4*1000/(spacing))<=max(ar_list):
    dia_h = int(input("Enter another trial value of diameter:"))
    spacing_h = int(input("Enter another trial value:"))
ast_hpro = math.pi*pow(dia_h,2)/4*(1000/(spacing_h))
print(ast_hpro)   

4487.9895051282765
Enter trial spacing:150
Enter a trial diameter:16
1340.4128655316451


Vertical Reinforcements

In [50]:
# check_ar = math.pi*(pow(dia,2)/4)*(1000/spacing)
ver_dia = 20 
ver_spc = 180
ast_vert = math.pi*pow(ver_dia,2)/4*(1000/(ver_spc))
print(ast_vert) 

1745.3292519943295


**Design of open foundation**

In [51]:
display(C2)

,Vertical Forces,Horizontal forces,Moment
Abutment self,124.600781,0.000000,691.538296
SS DL,5220.625000,0.000000,31976.328125
SS Live load,895.524000,0.000000,5485.084500
Surface DL,214.500000,0.000000,1313.812500
Foundation,1635.000000,0.000000,0.000000
Earth Hz (dry),0.000000,0.000000,0.000000
Earth Hz (wet),0.000000,173.845126,-394.411129
Earth ver,70.237990,0.000000,207.202070
Live load surcharge (Hz),0.000000,29.775551,-111.286123
Live load surcharge(Ver),81.000000,0.000000,340.200000


In [52]:
bas_1 = [1.35,1.35,1.5,1.75,1.35,1.5,1.5,1.5,1.2,1.2,1.15,.9,1.35,0,0,0,0,0,0]
bas_2 = [1,1,1.3,1,1,1.3,1.3,1.3,1,1,1,.8,1,0,0,0,0,0,0]
seis_com = [1.35,1.35,0.75,1.75,1.35,0,1,1,.2,.2,.2,.5,1.35,0,1.5,1.5,1.5,1.5,1.5]

#Basic combination 1
v_bas_1 = []
hz_bas_1 = []
mom_bas_1 = []
for index,i in enumerate(bas_1):
    v_bas_1.append(round(bas_1[index]*ver_force[index],5))
    hz_bas_1.append(round(bas_1[index]*h_forc[index],5))
    mom_bas_1.append(round(bas_1[index]*new_M[index],5))
X1 = pd.DataFrame({"Vertical Forces":v_bas_1, 'Horizontal forces':hz_bas_1,"Moment":mom_bas_1},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge(hz)','Live load Surcharge(ver)','Braking','Temperature','backfill','Braking uplift','SS DL(seismic)','Foundation(seismic)','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
X1.loc[len(X1.index)] = [X1['Vertical Forces'].sum(), X1['Horizontal forces'].sum(),X1['Moment'].sum()]
display(X1.rename(index={19:'Total'}))

#Basic combination 2
v_bas_2 = []
hz_bas_2 = []
mom_bas_2 = []
for index,i in enumerate(bas_2):
    v_bas_2.append(round(bas_2[index]*ver_force[index],5))
    hz_bas_2.append(round(bas_2[index]*h_forc[index],5))
    mom_bas_2.append(round(bas_2[index]*new_M[index],5))
X2 = pd.DataFrame({"Vertical Forces":v_bas_2, 'Horizontal forces':hz_bas_2,"Moment":mom_bas_2},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge(hz)','Live load Surcharge(ver)','Braking','Temperature','backfill','Braking uplift','SS DL(seismic)','Foundation(seismic)','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
X2.loc[len(X2.index)] = [X2['Vertical Forces'].sum(), X2['Horizontal forces'].sum(),X2['Moment'].sum()]
display(X2.rename(index={19:'Total'}))

#Seismic combination
v_seis = []
hz_seis = []
mom_seis = []
for index,i in enumerate(seis_com):
    v_seis.append(round(seis_com[index]*ver_force[index],5))
    hz_seis.append(round(seis_com[index]*h_forc[index],5))
    mom_seis.append(round(seis_com[index]*new_M[index],5))
X3 = pd.DataFrame({"Vertical Forces":v_seis, 'Horizontal forces':hz_seis,"Moment":mom_seis},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge(hz)','Live load Surcharge(ver)','Braking','Temperature','backfill','Braking uplift','SS DL(seismic)','Foundation(seismic)','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
X3.loc[len(X3.index)] = [X3['Vertical Forces'].sum(), X3['Horizontal forces'].sum(),X3['Moment'].sum()]
display(X3.rename(index={19:'Total'}))


,Vertical Forces,Horizontal forces,Moment
Abutment self,168.21105,0.00000,933.57670
SS DL,7047.84375,0.00000,43168.04297
SS Live load,1343.28600,0.00000,8227.62675
Surface DL,375.37500,0.00000,2299.17187
Foundation,2207.25000,0.00000,0.00000
Earth Hz (dry),0.00000,0.00000,0.00000
Earth Hz (wet),0.00000,260.76769,-591.61669
Earth ver,105.35698,0.00000,310.80311
Live load Surcharge(hz),0.00000,35.73066,-133.54335
Live load Surcharge(ver),97.20000,0.00000,408.24000


,Vertical Forces,Horizontal forces,Moment
Abutment self,124.60078,0.00000,691.53830
SS DL,5220.62500,0.00000,31976.32812
SS Live load,1164.18120,0.00000,7130.60985
Surface DL,214.50000,0.00000,1313.81250
Foundation,1635.00000,0.00000,0.00000
Earth Hz (dry),0.00000,0.00000,0.00000
Earth Hz (wet),0.00000,225.99866,-512.73447
Earth ver,91.30939,0.00000,269.36269
Live load Surcharge(hz),0.00000,29.77555,-111.28612
Live load Surcharge(ver),81.00000,0.00000,340.20000


,Vertical Forces,Horizontal forces,Moment
Abutment self,168.21105,0.00000,933.57670
SS DL,7047.84375,0.00000,43168.04297
SS Live load,671.64300,0.00000,4113.81338
Surface DL,375.37500,0.00000,2299.17187
Foundation,2207.25000,0.00000,0.00000
Earth Hz (dry),0.00000,0.00000,0.00000
Earth Hz (wet),0.00000,173.84513,-394.41113
Earth ver,70.23799,0.00000,207.20207
Live load Surcharge(hz),0.00000,5.95511,-22.25722
Live load Surcharge(ver),16.20000,0.00000,68.04000


In [53]:
moi_fdn = moi('rectangle', [9,7.85])[0]
print(moi_fdn)

362.8025
